<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220329_ji.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install gluonnlp
# !pip install mxnet
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install transformers
!pip install sentencepiece
!pip install kobert-transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-2df3wmu2/kobert-tokenizer_7c1977c3af8f4790aff9eaf4a6c2f4c5
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-2df3wmu2/kobert-tokenizer_7c1977c3af8f4790aff9eaf4a6c2f4c5
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-di0ty1tt
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-di0ty1tt


In [102]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer
from sklearn.model_selection import train_test_split
import logging

In [104]:
logging.basicConfig(level=logging.DEBUG)

In [105]:
train = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')

In [106]:
#train = train.sample(frac=0.1)

In [107]:
train

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [58]:
train['digit_1'].value_counts()

G    246472
I    187425
C    105192
S    100396
H     98038
P     46610
L     40140
Q     36087
F     35050
R     29751
M     28434
N     17701
J     10862
K     10378
O      2965
E      2255
A      1064
D       756
B       424
Name: digit_1, dtype: int64

In [59]:
train.isnull().sum()

AI_id            0
digit_1          0
digit_2          0
digit_3          0
text_obj     16677
text_mthd    43619
text_deal    67652
dtype: int64

In [60]:
train = train.fillna("")

In [61]:
train.isnull().sum()

AI_id        0
digit_1      0
digit_2      0
digit_3      0
text_obj     0
text_mthd    0
text_deal    0
dtype: int64

In [62]:
df = train.reset_index(drop=True)

In [63]:
#df = train[train['digit_2']==47].copy().reset_index(drop=True)

In [64]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육
...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스


In [65]:
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df["text"] = df["text_obj"]+ " " + df["text_mthd"] + " " + df["text_deal"]

In [66]:
df['target'].nunique()

225

In [67]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,S 95 952,카센터에서 자동차부분정비 타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,G 47 472,상점내에서 일반인을 대상으로 채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,G 46 467,절단하여사업체에도매 공업용고무를가지고 합성고무도매
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,G 47 475,영업점에서 일반소비자에게 열쇠잠금장치
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,Q 87 872,어린이집 보호자의 위탁을 받아 취학전아동보육
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C 13 134,제품입고 워싱 청바지워싱
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F 42 424,현장에서 고객의요청에의해 실내인테리어
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G 47 474,영업점에서 일반소비자에게 여성의류 판매
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P 85 856,사업장에서 일반고객을대상으로 필라테스


In [68]:
#df['text'] = df['text'].str.strip()

In [69]:
df['No.']=df.index

In [70]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,S 95 952,카센터에서 자동차부분정비 타이어오일교환,0
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,G 47 472,상점내에서 일반인을 대상으로 채소.과일판매,1
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,G 46 467,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,G 47 475,영업점에서 일반소비자에게 열쇠잠금장치,3
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,Q 87 872,어린이집 보호자의 위탁을 받아 취학전아동보육,4
...,...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C 13 134,제품입고 워싱 청바지워싱,999995
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F 42 424,현장에서 고객의요청에의해 실내인테리어,999996
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G 47 474,영업점에서 일반소비자에게 여성의류 판매,999997
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P 85 856,사업장에서 일반고객을대상으로 필라테스,999998


In [71]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [72]:
def text_to_token(df:pd.DataFrame):
  input_ids = list()
  token_type_ids = list()
  attention_mask = list()

  for i in range(len(df)):
    temp = tokenizer.encode_plus(df['text'].loc[i], add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask = True, 
            return_token_type_ids=True,
            truncation=True)
    input_ids.append(temp['input_ids'])
    token_type_ids.append(temp['token_type_ids'])
    attention_mask.append(temp['attention_mask'])

  return input_ids, token_type_ids, attention_mask

df['input_ids'], df['token_type_ids'], df['attention_mask'] = text_to_token(df)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [73]:
df[['input_ids', 'token_type_ids', 'attention_mask']].head()

,input_ids,token_type_ids,attention_mask
0,"[2, 4635, 6594, 3893, 6398, 6416, 7227, 6441, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
1,"[2, 2658, 7220, 5678, 6903, 3807, 7119, 7088, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,"[2, 4067, 5788, 7815, 6493, 6879, 6901, 6168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3358, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,"[2, 3235, 2400, 7167, 3565, 7088, 2226, 4598, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"


In [74]:
## GPU
device = torch.device("cuda:0")

In [75]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [76]:
my_dict = {}
for i in range(len(df['target'].unique())):
  my_dict[df['target'].unique()[i]] = i

In [77]:
my_dict

{'A 1 11': 174,
 'A 1 12': 197,
 'A 1 14': 188,
 'A 2 20': 171,
 'A 3 31': 199,
 'A 3 32': 179,
 'B 5 51': 223,
 'B 6 61': 209,
 'B 6 62': 207,
 'B 7 71': 121,
 'B 7 72': 80,
 'B 8 80': 218,
 'C 10 101': 97,
 'C 10 102': 10,
 'C 10 103': 88,
 'C 10 104': 100,
 'C 10 105': 135,
 'C 10 106': 72,
 'C 10 107': 7,
 'C 10 108': 178,
 'C 11 111': 148,
 'C 11 112': 110,
 'C 12 120': 224,
 'C 13 131': 158,
 'C 13 132': 107,
 'C 13 133': 177,
 'C 13 134': 103,
 'C 13 139': 111,
 'C 14 141': 43,
 'C 14 142': 215,
 'C 14 143': 180,
 'C 14 144': 122,
 'C 15 151': 91,
 'C 15 152': 182,
 'C 16 161': 95,
 'C 16 162': 115,
 'C 16 163': 67,
 'C 17 171': 108,
 'C 17 172': 138,
 'C 17 179': 129,
 'C 18 181': 81,
 'C 18 182': 204,
 'C 19 191': 195,
 'C 19 192': 203,
 'C 20 201': 151,
 'C 20 202': 90,
 'C 20 203': 56,
 'C 20 204': 79,
 'C 20 205': 157,
 'C 21 211': 206,
 'C 21 212': 187,
 'C 21 213': 201,
 'C 22 221': 18,
 'C 22 222': 33,
 'C 23 231': 156,
 'C 23 232': 120,
 'C 23 233': 94,
 'C 23 239': 85,

In [78]:
def target_to_label(text:str):
  return my_dict[text]

df['target_label'] = df['target'].map(target_to_label)

In [79]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.,input_ids,token_type_ids,attention_mask,target_label
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,S 95 952,카센터에서 자동차부분정비 타이어오일교환,0,"[2, 4635, 6594, 3893, 6398, 6416, 7227, 6441, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,G 47 472,상점내에서 일반인을 대상으로 채소.과일판매,1,"[2, 2658, 7220, 5678, 6903, 3807, 7119, 7088, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,G 46 467,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2,"[2, 4067, 5788, 7815, 6493, 6879, 6901, 6168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,G 47 475,영업점에서 일반소비자에게 열쇠잠금장치,3,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3358, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",3
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,Q 87 872,어린이집 보호자의 위탁을 받아 취학전아동보육,4,"[2, 3235, 2400, 7167, 3565, 7088, 2226, 4598, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C 13 134,제품입고 워싱 청바지워싱,999995,"[2, 4158, 7138, 5439, 3530, 6750, 4483, 6273, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",103
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F 42 424,현장에서 고객의요청에의해 실내인테리어,999996,"[2, 5063, 994, 7095, 6999, 7431, 6896, 7095, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",68
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G 47 474,영업점에서 일반소비자에게 여성의류 판매,999997,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3312, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",11
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P 85 856,사업장에서 일반고객을대상으로 필라테스,999998,"[2, 2609, 7184, 3807, 5440, 7088, 5816, 7078, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",12


In [80]:
df[['target', 'target_label']].head()

,target,target_label
0,S 95 952,0
1,G 47 472,1
2,G 46 467,2
3,G 47 475,3
4,Q 87 872,4


In [81]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.,input_ids,token_type_ids,attention_mask,target_label
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,S 95 952,카센터에서 자동차부분정비 타이어오일교환,0,"[2, 4635, 6594, 3893, 6398, 6416, 7227, 6441, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,G 47 472,상점내에서 일반인을 대상으로 채소.과일판매,1,"[2, 2658, 7220, 5678, 6903, 3807, 7119, 7088, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,G 46 467,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2,"[2, 4067, 5788, 7815, 6493, 6879, 6901, 6168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,G 47 475,영업점에서 일반소비자에게 열쇠잠금장치,3,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3358, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",3
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,Q 87 872,어린이집 보호자의 위탁을 받아 취학전아동보육,4,"[2, 3235, 2400, 7167, 3565, 7088, 2226, 4598, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C 13 134,제품입고 워싱 청바지워싱,999995,"[2, 4158, 7138, 5439, 3530, 6750, 4483, 6273, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",103
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F 42 424,현장에서 고객의요청에의해 실내인테리어,999996,"[2, 5063, 994, 7095, 6999, 7431, 6896, 7095, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",68
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G 47 474,영업점에서 일반소비자에게 여성의류 판매,999997,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3312, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",11
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P 85 856,사업장에서 일반고객을대상으로 필라테스,999998,"[2, 2609, 7184, 3807, 5440, 7088, 5816, 7078, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",12


In [82]:
# dataset_train = df[['No.', 'text', 'target_label']][:int(len(df)*0.8)]
# dataset_test = df[['No.', 'text', 'target_label']][int(len(df)*0.8):]
dataset_train, dataset_test = train_test_split(df[['No.', 'text', 'target_label']], test_size=0.2, random_state=42, shuffle=True)
dataset_train.to_csv('dataset_train.txt', index=False, header=True, sep='\t', encoding='utf8')
dataset_test.to_csv('dataset_test.txt', index=False, header=True, sep='\t', encoding='utf8')

In [83]:
dataset_train = nlp.data.TSVDataset('/content/dataset_train.txt', field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset('/content/dataset_test.txt', field_indices=[1,2], num_discard_samples=1)

In [84]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [85]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [86]:
## Setting parameters
max_len = 256
batch_size = 8
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [87]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [88]:
next(iter(data_train))

(array([   2, 2068, 7191, 6234, 2184, 1083, 7239, 4152,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [89]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [90]:
len(df['target_label'].unique())

225

In [91]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=520,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [92]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [93]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [94]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [95]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [96]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [97]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [98]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    # torch.cuda.empty_cache()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/100000 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 6.501279354095459 train acc 0.0


KeyboardInterrupt: ignored

In [99]:
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,target,text,No.,input_ids,token_type_ids,attention_mask,target_label
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,S 95 952,카센터에서 자동차부분정비 타이어오일교환,0,"[2, 4635, 6594, 3893, 6398, 6416, 7227, 6441, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,G 47 472,상점내에서 일반인을 대상으로 채소.과일판매,1,"[2, 2658, 7220, 5678, 6903, 3807, 7119, 7088, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,G 46 467,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2,"[2, 4067, 5788, 7815, 6493, 6879, 6901, 6168, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",2
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,G 47 475,영업점에서 일반소비자에게 열쇠잠금장치,3,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3358, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",3
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,Q 87 872,어린이집 보호자의 위탁을 받아 취학전아동보육,4,"[2, 3235, 2400, 7167, 3565, 7088, 2226, 4598, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,C 13 134,제품입고 워싱 청바지워싱,999995,"[2, 4158, 7138, 5439, 3530, 6750, 4483, 6273, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",103
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,F 42 424,현장에서 고객의요청에의해 실내인테리어,999996,"[2, 5063, 994, 7095, 6999, 7431, 6896, 7095, 7...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",68
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,G 47 474,영업점에서 일반소비자에게 여성의류 판매,999997,"[2, 3382, 7220, 6903, 3807, 6611, 6897, 3312, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",11
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,P 85 856,사업장에서 일반고객을대상으로 필라테스,999998,"[2, 2609, 7184, 3807, 5440, 7088, 5816, 7078, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",12


In [101]:
df['input_ids'].map(len).max()

64

In [ ]:
max(df['text'].map(len))

In [ ]:
list(map(len,df['text']))

In [ ]:
len('dsads')

In [ ]:
len